### Матрица признаков для текстов       

Построим матрицу признаков для текстов  с использованием словаря и вектора весов, полученного ранее. Будем использовать взвешивание 
$$lTFIDF = \ln(TF + 1) \cdot IDF$$

Значения признаков будим маштабировать так, чтобы для каждого признака его среднее значение по выборке равнялось 0, а среднеквадратичное отклонение 
1: 
$$x^{scaled}_{i} = \frac{x_{i} - E(x)} {\sigma(x)}$$


В результате масштабирования для каждого столбца матрицы признаков среднее будет равняться 0, а среднеквадратичное отклонение 1. При расчёте среднеквадратического отклонения будем использовать скорректированную оценку 

$$\sigma=\sqrt{\frac{\sum_{i-1}^n(x_i - E(x))^2}{n - 1}}$$.

Чтобы получить такую оценку с помощью numpy, необходимо передать параметр ddof=1: 

feature_matrix = np.zeros((num_docs, num_feats))    
feats_std = feature_matrix.std(0, ddof=1)

In [1]:
import numpy as np
import collections
import re
from dlnlputils.data import build_vocabulary

In [2]:
TOKEN_RE = re.compile(r'[\w\d]+')
def tokenize_text_simple_regex(txt, min_token_size=1):
    txt = txt.lower()
    all_tokens = TOKEN_RE.findall(txt)
    return [token for token in all_tokens if len(token) >= min_token_size]

def tokenize_corpus(texts, tokenizer=tokenize_text_simple_regex, **tokenizer_kwargs):
    return [tokenizer(text, **tokenizer_kwargs) for text in texts]

In [3]:
texts = ["Казнить нельзя, помиловать. Нельзя наказывать.", "Казнить, нельзя помиловать. Нельзя освободить.", "Нельзя не помиловать.", "Обязательно освободить."]
texts_tokenized = tokenize_corpus(texts)
print(' '.join(texts_tokenized[2]))
MAX_DF = 1
MIN_COUNT = 0
vocab, w_doc_freq = build_vocabulary(texts_tokenized, max_doc_freq=MAX_DF, min_count=MIN_COUNT)

нельзя не помиловать


In [4]:
dd = {x[0]: w_doc_freq[x[1]] for x in vocab.items()}
dd

{'помиловать': 0.75,
 'нельзя': 0.75,
 'казнить': 0.5,
 'освободить': 0.5,
 'наказывать': 0.25,
 'не': 0.25,
 'обязательно': 0.25}

In [5]:
e = sorted(dd.items(), key=lambda pair: (pair[1],pair[0]))
print(e)

[('наказывать', 0.25), ('не', 0.25), ('обязательно', 0.25), ('казнить', 0.5), ('освободить', 0.5), ('нельзя', 0.75), ('помиловать', 0.75)]


In [6]:
new_vocab = {}
for x in list(vocab.keys()):
    for i,y in enumerate(e):
        if x == y[0]:
            new_vocab[x] = i

new_vocab

{'помиловать': 6,
 'нельзя': 5,
 'казнить': 3,
 'освободить': 4,
 'наказывать': 0,
 'не': 1,
 'обязательно': 2}

In [7]:
for _,w in e:
    print(w/len(e))

0.03571428571428571
0.03571428571428571
0.03571428571428571
0.07142857142857142
0.07142857142857142
0.10714285714285714
0.10714285714285714


In [8]:
#lTFIDF=ln(TF+1)⋅IDF.
import scipy
def vectorize_texts_MY(tokenized_texts, word2id, word2freq, mode='tfidf', scale=True):
    assert mode in {'tfidf', 'idf', 'tf', 'bin'}

    # считаем количество употреблений каждого слова в каждом документе
    result = scipy.sparse.dok_matrix((len(tokenized_texts), len(word2id)), dtype='float32')
    for text_i, text in enumerate(tokenized_texts):
        for token in text:
            if token in word2id:
                result[text_i, word2id[token]] += 1

    # получаем бинарные вектора "встречается или нет"
    if mode == 'bin':
        result = (result > 0).astype('float32')

    # получаем вектора относительных частот слова в документе
    elif mode == 'tf':
        result = result.tocsr()
        result = result.multiply(1 / result.sum(1))

    # полностью убираем информацию о количестве употреблений слова в данном документе,
    # но оставляем информацию о частотности слова в корпусе в целом
    elif mode == 'idf':
        result = (result > 0).astype('float32').multiply(1 / word2freq)

    # учитываем всю информацию, которая у нас есть:
    # частоту слова в документе и частоту слова в корпусе
    elif mode == 'tfidf':
        result = result.tocsr().toarray()
        result = np.log(result*(1 / result.sum(1)).reshape([4,1]) + 1)  # разделить каждую строку на её длину
        result = result*(1 / word2freq)  # разделить каждый столбец на вес слова

    # if scale:
    #     result = result.tocsc()
    #     result -= result.min()
    #     result /= (result.max() + 1e-6)

    return result

In [9]:
VECTORIZATION_MODE = 'tfidf'
word2freq = w_doc_freq[::-1]
texts_vectors = vectorize_texts_MY(texts_tokenized, new_vocab, word2freq, mode=VECTORIZATION_MODE)

In [10]:
texts_tokenized

[['казнить', 'нельзя', 'помиловать', 'нельзя', 'наказывать'],
 ['казнить', 'нельзя', 'помиловать', 'нельзя', 'освободить'],
 ['нельзя', 'не', 'помиловать'],
 ['обязательно', 'освободить']]

In [11]:
arr = texts_vectors
L = (arr - arr.mean(axis=0))/arr.std(0, ddof=1)

In [12]:
arr.mean(axis=0)

array([0.18232161, 0.28768212, 0.4054651 , 0.18232161, 0.29389334,
       0.32020888, 0.21744178], dtype=float32)

In [13]:
# ОТВЕТ
for i in range(L.shape[0]):
    print(' '.join(map(str,L[i])))

1.5 -0.5 -0.5 0.86602545 -0.7630126 0.5954669 0.16096799
-0.5 -0.5 -0.5 0.86602545 0.18368244 0.5954669 0.16096799
-0.5 1.5 -0.5 -0.86602545 -0.7630126 0.29382402 1.0424348
-0.5 -0.5 1.5 -0.86602545 1.3423429 -1.4847578 -1.3643708


In [14]:
arr.mean(axis=0)

array([0.18232161, 0.28768212, 0.4054651 , 0.18232161, 0.29389334,
       0.32020888, 0.21744178], dtype=float32)

In [15]:
feature_matrix = np.zeros((len(L), len(L[0])))
np.array([4,4])/np.array([2,2])

array([2., 2.])